# Sessão síncrona de comunicação segura entre dois agentes

**Construção de uma sessão síncrona de comunicação segura entre dois agentes (o Emitter e o Receiver), combinando os seguintes elementos constituintes:**
- **a cifra simétrica TAES  (AES  na  sua versão tweak) usando autenticação do criptograma  em cada superbloco.**
- **o protocolo de acordo de chaves Diffie-Hellman com verificação da chave, e  autenticação dos agentes através do esquema de assinaturas DSA.**

**Use chaves DH/DSA de 3072 bits para acordo de chaves e autenticação dos agentes. Tenha em muita atenção os  tempos de vida diferentes para as chaves acordadas (chaves de sessão) e as chaves no esquema de assinaturas (chaves “permanentes”).**

In [1]:
from multiprocessing import Process, Pipe
from getpass import getpass

class Connection:
    def __init__(self, left, right, timeout=None):
        left_end, right_end = Pipe()
        self.timeout = timeout
        self.lproc = Process(target = left, args=(left_end,))
        self.rproc = Process(target = right, args=(right_end,))
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join(self.timeout)
    def manual(self):
        self.left()
        self.right()

## Receiver e Emitter (EDITARRRR)

A comunicação entre dois agentes com recurso à classe anteriormente definida é feita por meio de duas funções
`Emitter` e `Receiver` que enviam e recebem, respetivamente, uma mensagem de cada vez que são invocadas.
Ambos os intervenientes devem introduzir uma palavra passe que permitirá a posterior autenticação e cifragem
dos dados trocados entre os mesmos. Esta palavra passe é depois processada por um mecanismo de derivação de chaves
conhecido como *Password-Based Key Derivation Function* (**PBDKF**) que permite obter chaves de 
tamanho arbitrário a partir de dados de tamanho reduzido que combina com um valor aleatório conhecido como *salt*.

Por forma a garantir a confidenficialidade do conteúdo das mensagens bem como a autenticidade e integridade destas
e de dados associados (metadados) foi usada a cifra simétrica AES em modo GCM (*Gallois/Counter Mode*) que permite a 
autenticação de dados adicionais invocando o método:

```
cipher.authenticate_additional_data(header.encode("utf-8"))
```

Esta autenticação é feita via uma *tag* que deve ser introduzida no momento de decifragem dos dados e, caso não corresponda ao valor esperado, resulta numa exceção.

Para além da mensagem trocada entre os dois interveniente são ainda enviados dados que permitem a sua decifragem
por parte do `Receiver` nomeadamente o IV (*Initialization Vector*) usado na cifragem dos dados, a *tag* esperada
para decifrar os mesmos e ainda os metadados previamente autenticados:

```
payload = {'header': header, 'content': content, 'iv':iv, 'tag': cipher.tag}
        
conn.send(payload)
```

In [3]:
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import (Cipher, algorithms, modes)
import os
from cryptography.hazmat.primitives.serialization import Encoding
from cryptography.hazmat.primitives.serialization import ParameterFormat

backend = default_backend()
cred_salt = os.urandom(16)

#TODO: tweak
tweak = os.urandom(16)

def Receiver(conn):
    pwd = getpass('Reciever password: ')
    try:
        #TODO: key exchange
        #TODO: hash tweak . SHA256
        payload = conn.recv()
        iv = payload['iv']
        tag = payload['tag']
        header = payload['header']
        ctxt = payload['content']
        #TODO: AES in CFB
        decryptor = Cipher(algorithms.AES(key),modes.GCM(iv, tag),backend).decryptor()
        decryptor.authenticate_additional_data(header)
        #TODO: XOR w/ cryptogram & hash tweak
        print("Received: "+(decryptor.update(ctxt) + decryptor.finalize()).decode("utf-8"))
        #TODO: XOR w/ msg & hash tweak 
        conn.close()
    except:
        print("Receiver: Erro na autenticação da mensagem")
        
def Emitter(conn):
    pwd = getpass('Emitter password: ')
    try:
        #TODO: key exchange DH
        parameters = dh.generate_parameters(generator=5, key_size=2048,backend=default_backend())
        new_msg = parameters.parameter_bytes(Encoding.PEM,ParameterFormat.PKCS3)
        key = parameters.generate_private_key()
        iv = os.urandom(12)
        #TODO: AES in CFB
        #TODO: hash tweak . SHA256
        cipher = Cipher(algorithms.AES(key),modes.CFB(iv),backend).encryptor()
        header = 'msgheader'.encode("utf-8")
        content = 'Here is a sample message for testing'
        h = hmac.HMAC(tweak, hashes.SHA256(), backend=default_backend())
        #TODO: XOR w/ msg & hash tweak 
        #TODO: XOR w/ cryptogram & hash tweak
        content_new = cipher.update(key + content.encode("utf-8") + h) + cipher.finalize()
        payload = {'header': header, 'content': content, 'iv':iv, 'tag': cipher.tag}
        conn.send(payload)
        conn.close()
    except:
       print("Emitter: Error")

IndentationError: unexpected unindent (<ipython-input-3-2076946004e6>, line 33)

In [6]:
Connection(Emitter, Receiver, timeout=10).manual()

Emitter password: ········
Reciever password: ········
Received: Here is a sample message for testing
